In [ ]:
from mtcnn import MTCNN
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
def crop_image(image, box):
    # Input: image: np array of pixels to crop
    #        box: a dictionary returned by detect_faces() containing coordinates/width/height of the box outlining the detected face
    # Output: an np array of pixels representing the cropped image
    x, y, width, height = box[0], box[1], box[2], box[3] # x,y,w,h of the box containing a detected face
    return image[y:y+height, x:x+width]

In [31]:
def resize_grayscale_image(image, size=(224,224)):
    return np.array(Image.fromarray(np.squeeze(image, axis=-1)).resize(size))

In [4]:
def resize_rgb_image(image, size=(224, 224)):
    return np.array((Image.fromarray(image)).resize(size))

(Option 1): Use images from a directory

In [ ]:
# Option 1: Use folders of grayscale images
pixels = []

mapping = {"Angry":0, "Disgust":1, "Fear":2, "Happy":3, "Sad":4, "Surprise":5, "Neutral":6}
emotions = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Neutral"]

for emotion in emotions:
    for file_path in glob.glob(f"Test v2\\{emotion}\\" + "*.jpg"):
        pixels.append([np.asarray(Image.open(file_path)), emotion])

detector = MTCNN()
cropped_images = []

for img in pixels:
    emotion = img[1]
    original_pxs = np.expand_dims(img[0], -1)
    pxs = np.tile(original_pxs, [1,1,3]) # MTCNN library only works with imgs with 3 channels per pixel. This pads the img to make the img have 3 channels
    output = detector.detect_faces(pxs)
    if (len(output) == 1):
        resized_image = resize_grayscale_image(crop_image(original_pxs, output[0]["box"]))
        cropped_images.append([emotion, resized_image])

In [ ]:
# Option 2: Use folders of RGB images
pixels = []

mapping = {"Angry":0, "Disgust":1, "Fear":2, "Happy":3, "Sad":4, "Surprise":5, "Neutral":6}
emotions = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Neutral"]

for emotion in emotions:
    for file_path in glob.glob(f"Test Set\\{emotion}\\" + "*.jpg"):
        pixels.append([np.asarray(Image.open(file_path)), emotion])

detector = MTCNN()
cropped_images = []

for img in pixels:
    emotion = img[1]
    pxs = img[0]
    output = detector.detect_faces(pxs)
    if (len(output) == 1):
        resized_image = resize_rgb_image(crop_image(pxs, output[0]["box"]))
        cropped_images.append([emotion, resized_image])


(Option 2): Use images stored in a CSV file

In [2]:
df = pd.read_csv("Face Images.csv")

In [ ]:
# For each image in the df, use the mtcnn to detect the face, then crop the image to just the face portion, and then resize to a constant size of 224x224 px
cropped_images = []
count = 0
detector = MTCNN()

for index, row in df.iterrows():
    original_image = np.fromstring(row["pixels"], dtype=int, sep=" ").reshape(row["height"], row["width"], 1).astype("float32")
    img = np.tile(original_image, [1,1,3]) # this MTCNN library only works with colour images (imgs with 3 channels), but grayscale images have only 1 channel. This pads the img to make the image have 3 channels
    output = detector.detect_faces(img)
    if len(output) == 1: # 1 face detected
        resized_image = resize_grayscale_image(crop_image(original_image, output[0]["box"]))
        cropped_images.append([row["emotion"], resized_image])

Save Images

In [44]:
# Save the images to the directory corresponding to their labels
expressions = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}

count = 0 # arbitrary number used to ensure images are saved with a unique name

for image in cropped_images:
    label = image[0]
    im = image[1]
    # convert pixels to Image class
    # add .convert('L') to save RGB images as grayscale
    img = Image.fromarray(im)#.convert('L')

    img.save(f"FolderName/{label}/img_{count}.jpg", "JPEG")
    count += 1